In [1]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot

model = tf.keras.models.load_model('/home/ilham/ITS_2Oct/material/model/keras_model.h5')


2023-10-01 14:43:01.309463: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 14:43:01.359743: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 14:43:01.361261: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-01 14:43:02.316058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-01 14:43:03.440146: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-01 14:43:03.440417: W tensorflow/core/common_r

In [2]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp7y6itwyc/assets


INFO:tensorflow:Assets written to: /tmp/tmp7y6itwyc/assets
2023-10-01 14:43:24.553279: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-10-01 14:43:24.553354: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-10-01 14:43:24.554018: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp7y6itwyc
2023-10-01 14:43:24.574385: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-10-01 14:43:24.574429: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp7y6itwyc
2023-10-01 14:43:24.629661: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-10-01 14:43:24.654497: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-10-01 14:43:25.052242: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

In [3]:
import pathlib

tflite_models_dir = pathlib.Path("/home/ilham/ITS_2Oct/material/model/quant")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"keras_model.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

679096

In [4]:
import numpy as np
import cv2

# Load the TFLite model
tflite_model_path = '/home/ilham/ITS_2Oct/material/model/quant/keras_model.tflite'  # Replace with the path to your TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

image_path = '/home/ilham/ITS_2Oct/material/input/bulldog.jpg'  # Replace with the path to your image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB if needed

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
# Preprocess the image to match the input format of your model
# You should adjust this preprocessing based on your model's requirements
input_shape = input_details[0]['shape'][1:3]  # Get the input shape (height, width)
image = cv2.resize(image, (input_shape[1], input_shape[0]))  # Resize to match model input size
image = image / 255.0  # Normalize pixel values to [0, 1]
# image = np.expand_dims(image, axis=0)  # Add batch dimension


# Set the input tensor
input_image = np.expand_dims(image, axis=0).astype(input_details[0]["dtype"])
interpreter.set_tensor(input_details[0]['index'], input_image)

# Run inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Process and interpret the results as needed
# For example, if it's a classification model, find the predicted class
predicted_class = np.argmax(output_data, axis=1)
print("Bulldog") if predicted_class == 0 else print("Kucing")

Bulldog
